In [ ]:
from typing import List, Dict, Literal
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
endpoint = os.getenv("OPENAI_ENDPOINT")

client = OpenAI(
            api_key=api_key,
            base_url=endpoint
        )

In [ ]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    # A new method
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    # A new method
    def reset(self) -> None:
        self._messages = []

In [ ]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")
memory.get_messages()

In [ ]:
memory.reset()
memory.get_messages()

In [ ]:
def chat_with_tools(user_question:str=None, 
                    memory:Memory=None, 
                    model:str="gpt-4o-mini", 
                    temperature=0.0, 
                    tools=None)-> str:
    messages = [{"role": "user", "content": user_question}]
    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()        
    
    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools=tools, # Providing available tools to the model
    )
    
    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls # If the model decides to call a function
    
    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
    
    return ai_message

In [ ]:
chat_with_tools(
    "2 to the power of -5?",
    #model="gpt-3.5-turbo",
)

In [ ]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [ ]:
power(2, 3)

In [ ]:
power(2, -5)

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentatiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [ ]:
memory.get_messages()

In [ ]:
ai_message = chat_with_tools(
    "2 to the power of -5?",
    #model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [ ]:
memory.get_messages()

In [ ]:
memory.last_message()['tool_calls']

In [ ]:
tool_call_id = memory.last_message()['tool_calls'][0].id
tool_call_id

In [ ]:
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
args

In [ ]:
result = power(args["base"], args["exponent"])
result

In [ ]:
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)

In [ ]:
memory.get_messages()

In [ ]:
ai_message = chat_with_tools(
    #model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [ ]:
memory.get_messages()

In [ ]:
memory.last_message()["content"]